In [68]:
import json, os, configparser

parser = configparser.ConfigParser()
parser.read('../Data/config.ini')
    
with open(parser['embeddings']['zero_shot_classifier_probs'], 'r') as fs:
        probs = json.load(fs)

In [69]:
import numpy as np
import pandas as pd
from IPython.display import display

In [70]:
length_dict = {key: len(value) for key, value in probs.items()}
np.sum(list(length_dict.values()))

247729

In [71]:
text_list=['an image of tumoral ovarian tissue', 'an image of healthy ovarian tissue']

In [72]:
c0 = [v[0] for sub_dict in probs.values() for v in sub_dict.values()]
c1 = [v[1] for sub_dict in probs.values() for v in sub_dict.values()]

k = [f'{k}' for k, sub_dict in probs.items() for sk, v in sub_dict.items()]
sk = [f'{sk}' for k, sub_dict in probs.items() for sk, v in sub_dict.items()]

df = pd.DataFrame(
    data = {'0': c0, '1': c1, 'sample': k, 'patch': sk},
)

In [73]:
top_pred = df.groupby('sample').apply(lambda x: x.nlargest(10,['0']))
top_pred

0         1  \
sample                                                                         
TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a239... 5593   0.612058  0.387942   
                                                   5601   0.611075  0.388925   
                                                   5527   0.604007  0.395993   
                                                   5577   0.602154  0.397846   
                                                   5561   0.597161  0.402839   
...                                                            ...       ...   
TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86BA... 36221  0.574006  0.425994   
                                                   36155  0.573314  0.426686   
                                                   36087  0.572760  0.427240   
                                                   36113  0.572049  0.427951   
                                                   36249  0.571483  0.428517   

                                                                                                     sample  \
sample                                                                                                        
TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a239... 5593   TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a23...   
                                                   5601   TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a23...   
                                                   5527   TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a23...   
                                                   5577   TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a23...   
                                                   5561   TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a23...   
...                                                                                                     ...   
TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86BA... 36221  TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86B...   
                                                   36155  TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86B...   
                                                   36087  TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86B...   
                                                   36113  TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86B...   
                                                   36249  TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86B...   

                                                                           patch  
sample                                                                            
TCGA-04-1331-01A-01-BS1.27aaf831-a80b-4a55-a239... 5593    patch_87_embedding.pt  
                                                   5601    patch_94_embedding.pt  
                                                   5527    patch_88_embedding.pt  
                                                   5577    patch_67_embedding.pt  
                                                   5561    patch_33_embedding.pt  
...                                                                          ...  
TCGA-WR-A838-01A-01-TS1.E8CA96CD-A253-4090-86BA... 36221   patch_27_embedding.pt  
                                                   36155   patch_63_embedding.pt  
                                                   36087  patch_211_embedding.pt  
                                                   36113   patch_62_embedding.pt  
                                                   36249  patch_191_embedding.pt  

[8010 rows x 4 columns]

In [74]:
import json, os, configparser
import shutil


parser = configparser.ConfigParser()
parser.read('../Data/config.ini')
    
best_embeddings_dir = parser['embeddings']['best_embeddings_dir']
embeddings_dir = parser['embeddings']['embeddings_dir']

In [75]:
import os
import shutil

for sample, patch in top_pred[['sample', 'patch']].itertuples(index=False, name=None):
    try:
        os.makedirs(best_embeddings_dir, exist_ok=True)
        os.makedirs(os.path.join(best_embeddings_dir, sample), exist_ok=True)
        
        source_path = os.path.join(embeddings_dir, sample, patch)
        destination_path = os.path.join(best_embeddings_dir, sample, patch)  
        shutil.copy(source_path, destination_path)
        
    except Exception as e:
        print(f"Error copying file: {e}")
